In [0]:
spark.sql("USE CATALOG dev_silver_f1_catalog")
spark.sql("USE SCHEMA dev_silver_f1_schema")

spark.sql("""
CREATE TABLE IF NOT EXISTS user (
    user_id INT,
    month STRING,
    action STRING
)
USING DELTA
PARTITIONED BY (month)
LOCATION 'abfss://raw@databdls.dfs.core.windows.net/delta_sample'
""")

spark.sql("""
INSERT INTO user VALUES
    (1, 'January', 'login'),
    (2, 'February', 'logout'),
    (3, 'March', 'comment'),
    (4, 'April', 'like'),
    (3, 'May', 'share'),
    (6, 'June', 'upload'),
    (3, 'July', 'update'),
    (8, 'August', 'delete'),
    (3, 'September', 'register'),
    (10, 'October', 'purchase')
""")

In [0]:
spark.sql("USE CATALOG dev_silver_formula1_catalog")
spark.sql("USE SCHEMA processed_formula1_schema")
spark.sql("DELETE FROM user WHERE user_id=3")

In [0]:
# Time travel: see history of the 'user' Delta table
display(spark.sql("DESCRIBE HISTORY user"))

In [0]:
# using veriosn to see history
display(spark.read.format("delta").option("versionAsOf", 1).table("user"))
# display(spark.read.format("delta").option("versionAsOf", 2).table("user"))

In [0]:
# Using timestamp for history
display(spark.read.format("delta").option("timestampAsOf", "2025-09-15T23:22:59.000+00:00").table("user"))

In [0]:
spark.sql("RESTORE TABLE user TO VERSION AS OF 1")

In [0]:
display(spark.sql("select * from user"))


In [0]:
display(spark.conf.get("spark.databricks.delta.lastCommitVersionInSession"))
try:
    display(spark.conf.get("spark.databricks.delta.checkpointInterval"))
except Exception:
    pass
try:
    display(spark.conf.get("spark.databricks.delta.logRetentionDuration"))  # default 30 days
except Exception:
    pass
try:
    display(spark.conf.get("spark.databricks.delta.deletedFileRetentionDuration")) # default 7 days
except Exception:
    pass

In [0]:
# spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled", "false") # CONFIG_NOT_AVAILABLE
# spark.conf.set("spark.delta.retentionDurationCheck.enabled", "false") # CONFIG_NOT_AVAILABLE

spark.sql("VACUUM user RETAIN 0 HOURS")

# To VACUUM a Delta table with zero-hour retention, you must set this configuration at the cluster level (in the cluster's advanced Spark config), not at runtime in your notebook. 


In [0]:
# If you want to see what files would be deleted before running the vacuum:
display(spark.sql("VACUUM user DRY RUN"))

In [0]:
display(spark.read.format("delta").option("versionAsOf", 7).table("user"))

In [0]:
%sql
delete from user where user_id in (8,2)

In [0]:
%sql
merge into user as tgt
using (select * from user version as of 8) as src
on tgt.user_id = src.user_id
when not matched then
insert *